# Install import dependancies

In [1]:
!pip3 install torch torchvision torchaudio

In [8]:
!pip3 install transformers requests beautifulsoup4 pandas numpy

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate Model

In [13]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encode and Calucalte Sentiment

In [23]:
tokens = tokenizer.encode('this is amazing', return_tensors='pt')

In [24]:
result = model(tokens)

In [25]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.9988, -2.3604, -0.8269,  1.1844,  3.2305]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
int(torch.argmax(result.logits))+1

5

# Collect Reviews

In [27]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [31]:
results[0].text

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [32]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

# load Reviews into DataFrame and Score

In [33]:
import numpy as np
import pandas as pd

In [34]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [35]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,The food was delicious. The ricotta pancakes w...
3,Great place with delicious food and friendly s...
4,"Great service, lovely location, and really ama..."


In [36]:
df.tail()

,review
5,Great food amazing coffee and tea. Short walk ...
6,Ricotta hot cakes! These were so yummy. I ate ...
7,It was ok. Had coffee with my friends. I'm new...
8,We came for brunch twice in our week-long visi...
9,I came to Social brew cafe for brunch while ex...


In [39]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [42]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [45]:
sentiment_score(df['review'].iloc[1])

4

In [50]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [53]:
df['review']

0    Very cute coffee shop and restaurant. They hav...
1    Six of us met here for breakfast before our wa...
2    The food was delicious. The ricotta pancakes w...
3    Great place with delicious food and friendly s...
4    Great service, lovely location, and really ama...
5    Great food amazing coffee and tea. Short walk ...
6    Ricotta hot cakes! These were so yummy. I ate ...
7    It was ok. Had coffee with my friends. I'm new...
8    We came for brunch twice in our week-long visi...
9    I came to Social brew cafe for brunch while ex...
Name: review, dtype: object

In [54]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,The food was delicious. The ricotta pancakes w...,4
3,Great place with delicious food and friendly s...,5
4,"Great service, lovely location, and really ama...",5
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,4
9,I came to Social brew cafe for brunch while ex...,5
